# Introduction
---
The purpose of this notebook is to generate and compute the 1-edit distance between all the possible rooted and unlabeled trees given a fixed `depth` and `maximum_degree`.

## Notebook configuration
Configure several notebook configuration settings.

In [1]:
# Disable some warnings

import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr'

# Use full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

## Libraries / Packages
Import several useful packages for the notebook and configure some extra options.

In [17]:
# Miscellaneous libraries
import os
import time
import random
import itertools
import numpy as np
import pandas as pd
import networkx as nx

from tqdm.notebook import tqdm
from collections import defaultdict, Counter

# Setup some options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100


# Visualization
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.style.use('seaborn-whitegrid')

# Fetching data to analyze
---
Fetch all the `smoothness.csv` files generated and analyze them.

In [15]:
!ls "../data/synthetic/erdos_renyi/N100_n30_p0.1_1624230420/teacher_outputs/regression/GIN/num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__1624230793501/student_outputs/GIN"

num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231832278.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231834168.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231836049.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231837957.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231839844.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231841719.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231843613.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231845474.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231847332.pkl
num1_hidden32_residualFalse_jkFalse__bias0_lower-0.3_upper0.3__epochs10__1624231849179.pkl
smoothness_stats.csv


In [34]:
smoothness_filenames = [
    f"../data/synthetic/erdos_renyi/N100_n30_p0.1_1624230420/teacher_outputs/regression/GIN/{x}/student_outputs/GIN/smoothness_stats.csv"
    for x in os.listdir('../data/synthetic/erdos_renyi/N100_n30_p0.1_1624230420/teacher_outputs/regression/GIN')
    if all([c.isdigit() for c in x[-13:]])
]

smoothness_stats = pd.DataFrame()

for smoothness_filename in tqdm(smoothness_filenames):
    if smoothness_stats.empty:
        smoothness_stats = (pd.read_csv(smoothness_filename, header=[0, 1], index_col=0)
                              .reset_index()
                              .rename(columns={'index': 'model'}))
    else:
        smoothness_stats_ = (pd.read_csv(smoothness_filename, header=[0, 1], index_col=0)
                               .reset_index()
                               .rename(columns={'index': 'model'}))
        smoothness_stats = smoothness_stats.append(smoothness_stats_, ignore_index=True)

smoothness_stats

  0%|          | 0/1000 [00:00<?, ?it/s]

model               0  \
                                                       mean_smoothness   
0    num1_hidden32_residualFalse_jkFalse__bias0_low...    29917.564731   
1    num1_hidden32_residualFalse_jkFalse__bias0_low...    27393.097139   
2    num1_hidden32_residualFalse_jkFalse__bias0_low...    43373.942505   
3    num1_hidden32_residualFalse_jkFalse__bias0_low...    29186.499014   
4    num1_hidden32_residualFalse_jkFalse__bias0_low...    12513.084718   
..                                                 ...             ...   
995  num1_hidden32_residualFalse_jkFalse__bias0_low...    36931.691408   
996  num1_hidden32_residualFalse_jkFalse__bias0_low...    16619.593367   
997  num1_hidden32_residualFalse_jkFalse__bias0_low...    20579.027700   
998  num1_hidden32_residualFalse_jkFalse__bias0_low...    19443.593212   
999  num1_hidden32_residualFalse_jkFalse__bias0_low...    17754.586495   

                                                            1  \
    std_dev_smoothness mean_rmse std_dev_rmse mean_smoothness   
0         45552.636277  0.340084     0.222330    20927.569494   
1         27870.977791  0.236785     0.135387    95253.630312   
2         44724.518275  0.373355     0.250598    13223.169663   
3         44406.243668  0.307858     0.167890    33040.670781   
4         11515.151046  0.227968     0.065278   165619.103410   
..                 ...       ...          ...             ...   
995       38452.428946  0.256899     0.132280    32161.052398   
996       16446.823309  0.324774     0.227502    13623.277619   
997       19324.238709  0.456593     0.248392     4798.094077   
998       27076.508668  0.462102     0.239599     6951.284086   
999       24932.491522  0.573732     0.322920     2698.062056   

                                                            2  \
    std_dev_smoothness mean_rmse std_dev_rmse mean_smoothness   
0          3201.721415  0.014253     0.014656    20565.519544   
1         17198.020015  0.017896     0.018484    91694.474426   
2          2475.184053  0.015190     0.021941    13898.251231   
3          8019.629047  0.022374     0.021787    32817.475151   
4         14109.282370  0.009021     0.005855   167254.282705   
..                 ...       ...          ...             ...   
995        2843.049728  0.009508     0.007748    33825.904061   
996        2618.063976  0.020127     0.012223    14171.186918   
997        1043.929156  0.015629     0.020802     4351.209057   
998        1630.983506  0.024893     0.019575     7215.600528   
999         857.276636  0.032844     0.036090     3436.764289   

                                                            3  \
    std_dev_smoothness mean_rmse std_dev_rmse mean_smoothness   
0          3357.551258  0.012115     0.017436    21323.508985   
1          5807.818178  0.005516     0.006009    92663.689692   
2           566.293545  0.004418     0.003924    13854.077968   
3          4258.779260  0.012863     0.012755    31341.001172   
4         16872.539172  0.010114     0.008969   160588.979297   
..                 ...       ...          ...             ...   
995        4662.744968  0.013654     0.014685    33249.892899   
996        1143.683164  0.007680     0.009519    13607.022308   
997         817.177297  0.015719     0.019608     4708.306010   
998         624.094718  0.008097     0.007584     7727.169340   
999         263.841256  0.010701     0.007005     3529.297285   

                                                            4  \
    std_dev_smoothness mean_rmse std_dev_rmse mean_smoothness   
0          2287.199654  0.009044     0.011144    20120.342354   
1          5641.788971  0.005588     0.005384    92993.411589   
2          2210.196528  0.014267     0.013244    14447.985657   
3          5095.635443  0.011284     0.016296    33219.237894   
4         27792.858457  0.014398     0.017128   170650.208537   
..                 ...       ...          ...             ...   
995        3744.459781  0.01

In [38]:
smoothness_stats.groupby('model').mean()

/Users/berges/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


0  \
                                                   mean_smoothness   
model                                                                
num1_hidden32_residualFalse_jkFalse__bias0_lowe...    30913.654545   

                                                                       \
                                                   std_dev_smoothness   
model                                                                   
num1_hidden32_residualFalse_jkFalse__bias0_lowe...       42808.170602   

                                                                           \
                                                   mean_rmse std_dev_rmse   
model                                                                       
num1_hidden32_residualFalse_jkFalse__bias0_lowe...  1.480869     1.047881   

                                                                 1  \
                                                   mean_smoothness   
model                                                                
num1_hidden32_residualFalse_jkFalse__bias0_lowe...    30248.803071   

                                                                       \
                                                   std_dev_smoothness   
model                                                                   
num1_hidden32_residualFalse_jkFalse__bias0_lowe...        4307.827654   

                                                                           \
                                                   mean_rmse std_dev_rmse   
model                                                                       
num1_hidden32_residualFalse_jkFalse__bias0_lowe...  0.032396     0.031909   

                                                                 2  \
                                                   mean_smoothness   
model                                                                
num1_hidden32_residualFalse_jkFalse__bias0_lowe...    30019.170746   

                                                                       \
                                                   std_dev_smoothness   
model                                                                   
num1_hidden32_residualFalse_jkFalse__bias0_lowe...        2876.876692   

                                                                           \
                                                   mean_rmse std_dev_rmse   
model                                                                       
num1_hidden32_residualFalse_jkFalse__bias0_lowe...  0.013438      0.01533   

                                                                 3  \
                                                   mean_smoothness   
model                                                                
num1_hidden32_residualFalse_jkFalse__bias0_lowe...    30319.278807   

                                                                       \
                                                   std_dev_smoothness   
model                                                                   
num1_hidden32_residualFalse_jkFalse__bias0_lowe...        3379.296655   

                                                                           \
                                                   mean_rmse std_dev_rmse   
model                                                                       
num1_hidden32_residualFalse_jkFalse__bias0_lowe...   0.01248     0.013213   

                                                                 4  \
                                                   mean_smoothness   
model                                                                
num1_hidden32_residualFalse_jkFalse__bias0_lowe...    30290.969551   

                                                                       \
                                                   std_dev_smoothness   
model                                                                   
num1_hidden32_residualFalse_jkFal